# ImSpiRE: <ins>Im</ins>age-aided <ins>Sp</ins>at<ins>i</ins>al <ins>R</ins>esolution <ins>E</ins>nhancement

## This is a tutorial written using Jupyter Notebook.

### Step 1. ImSpiRE installation following the [tutorial](https://github.com/Yizhi-Zhang/ImSpiRE).

### Step 2. Input preparation

ImSpiRE utilizes the count file in tab-delimited format or hierarchical-data format (HDF5 or H5) and the image file in TIFF format, as well as a file containing spot coordinates as input. 

We provided a small [test dataset](https://github.com/Yizhi-Zhang/ImSpiRE/tree/master/test/test_data) containing the raw count matrix, image and spot coordinates. A CellProfiler pipeline is also included in the test dataset for use if required.

### Step 3. Operation of ImSpiRE

### 3.1 Load required packages

In [ ]:
import imspire_object as imspire
import pandas as pd
import numpy as np
import scanpy as sc

### 3.2 Custom parameters

In [ ]:
imspire_param=imspire.ImSpiRE_Parameters()

In [ ]:
imspire_param.BasicParam_InputCountFile="test_data/count_matrix.tsv"
imspire_param.BasicParam_InputDir="test_data/"
imspire_param.BasicParam_InputImageFile="test_data/image.tif"
imspire_param.BasicParam_PlatForm="ST"
imspire_param.BasicParam_Mode=2
imspire_param.BasicParam_OutputName="test_output"
imspire_param.BasicParam_Overwriting=True
imspire_param.CellProfilerParam_Pipeline="test_data/Cellprofiler_Pipeline_HE.cppipe"

### 3.3 Run ImSpiRE

#### 3.3.1 step by step

In [ ]:
imspire.create_folder(imspire_param.BasicParam_OutputDir,
                      imspire_param.BasicParam_OutputName,
                      imspire_param.BasicParam_Overwriting)

In [ ]:
imdata=imspire.ImSpiRE_Data()
imdata.read_ST(imspire_param.BasicParam_InputDir, count_file=imspire_param.BasicParam_InputCountFile)

In [ ]:
## optional
imdata.preprocess(min_counts=imspire_param.Threshold_MinCounts, 
                  max_counts=imspire_param.Threshold_MaxCounts, 
                  pct_counts_mt=imspire_param.Threshold_MitoPercent, 
                  min_cells=imspire_param.Threshold_MinSpot)

In [ ]:
im=imspire.ImSpiRE_HE_Image(imspire_param.BasicParam_InputImageFile,
                            imspire_param.BasicParam_PlatForm,
                            imspire_param.BasicParam_OutputDir,
                            imspire_param.BasicParam_OutputName,
                            imspire_param.FeatureParam_IterCount)

In [ ]:
spot_image_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/ImageResults/SpotImage"
im.segment_spot_image(pos_in_tissue_filter=imdata.pos_in_tissue_filter,
                      output_path=spot_image_output_path,
                      crop_size=imspire_param.ImageParam_CropSize)

In [ ]:
patch_image_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/ImageResults/PatchImage"
im.generate_patch_locations_2(pos_in_tissue=imdata.pos_in_tissue_filter,
                              dist=imspire_param.ImageParam_PatchDist)
im.segment_patch_image(patch_in_tissue=im.patch_in_tissue, 
                       output_path=patch_image_output_path, 
                       crop_size=imspire_param.ImageParam_CropSize)

In [ ]:
spot_cp=imspire.ImSpiRE_HE_CellProfiler(image_file_path=f"{spot_image_output_path}", 
                                        cellprofiler_pipeline=imspire_param.CellProfilerParam_Pipeline)
patch_cp=imspire.ImSpiRE_HE_CellProfiler(image_file_path=f"{patch_image_output_path}", 
                                         cellprofiler_pipeline=imspire_param.CellProfilerParam_Pipeline)

In [ ]:
spot_feature_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/FeatureResults/SpotFeature"
spot_cp.compute_image_features(output_path=spot_feature_output_path,
                               number_of_kernels=imspire_param.CellProfilerParam_KernelNumber)
spot_cp.filter_image_features(output_path=spot_feature_output_path)

In [ ]:
patch_feature_output_path=f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/FeatureResults/PatchFeature"
patch_cp.compute_image_features(output_path=patch_feature_output_path,
                                number_of_kernels=imspire_param.CellProfilerParam_KernelNumber)
patch_cp.filter_image_features(output_path=patch_feature_output_path)

In [ ]:
spot_features=spot_cp.image_features.loc[imdata.pos_in_tissue_filter.index,]

patch_cp.image_features.index=patch_cp.image_features.index.astype('int')
patch_features=patch_cp.image_features.sort_index()
patch_features.index=list(range(patch_features.shape[0]))

spot_features.to_csv(f"{spot_feature_output_path}/Image_Features_Image_filter.txt", sep = "\t")
patch_features.to_csv(f"{patch_feature_output_path}/Image_Features_Image_filter.txt", sep = "\t")

In [ ]:
spot_locations=np.array(imdata.pos_in_tissue_filter.loc[:,["pxl_row_in_fullres","pxl_col_in_fullres"]])
patch_locations=np.array(im.patch_in_tissue.loc[:,["pxl_row","pxl_col"]])

In [ ]:
spot_feature=pd.read_csv(f"{spot_feature_output_path}/Image_Features_Image_filter.txt",sep="\t",index_col=0)
patch_feature=pd.read_csv(f"{patch_feature_output_path}/Image_Features_Image_filter.txt",sep="\t",index_col=0)

## make sure spot features and patch features have the same dimension
spot_feature=spot_feature.dropna(axis=1)
patch_feature=patch_feature.dropna(axis=1)
commom_feature=list(set(spot_feature.columns).intersection(set(patch_feature.columns)))
spot_feature = spot_feature.loc[:,commom_feature]
patch_feature = patch_feature.loc[:,commom_feature]

spot_feature=np.array(spot_feature.loc[imdata.pos_in_tissue_filter.index,])
patch_feature=np.array(patch_feature.sort_index())

In [ ]:
exp_data=imdata.adata.to_df()
exp_data=exp_data.loc[imdata.pos_in_tissue_filter.index,]

In [ ]:
ot_solver=imspire.ImSpiRE_OT_Solver(spot_locations,patch_locations,
                                    spot_image_features=spot_feature,
                                    patch_image_features=patch_feature,
                                    spot_gene_expression=exp_data,
                                    random_state=imspire_param.BasicParam_RandomState)

In [ ]:
ot_solver.setup_cost_matrices(alpha=imspire_param.OptimalTransportParam_Alpha,
                              num_neighbors=imspire_param.OptimalTransportParam_NumNeighbors)
ot_solver.solve_OT(beta=imspire_param.OptimalTransportParam_Beta, 
                   epsilon=imspire_param.OptimalTransportParam_Epsilon,
                   numItermax=imspire_param.OptimalTransportParam_NumIterMax,
                   verbose=imspire_param.BasicParam_Verbose)

In [ ]:
exp_data_hr=imspire.compute_high_resolution_expression_profiles(exp_data,ot_solver.T)

In [ ]:
## output results
im.patch_in_tissue.index=list(range(im.patch_in_tissue.shape[0]))
im.patch_in_tissue.to_csv(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/{imspire_param.BasicParam_OutputName}_PatchLocations.txt", sep = "\t")

adata_hr=sc.AnnData(exp_data_hr)
adata_hr.write_h5ad(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/{imspire_param.BasicParam_OutputName}_ResolutionEnhancementResult.h5ad")

np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_M_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.M)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_C1_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.C1)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_C2_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.C2)
np.save(f"{imspire_param.BasicParam_OutputDir}/{imspire_param.BasicParam_OutputName}/SupplementaryResults/ot_T_alpha{imspire_param.OptimalTransportParam_Alpha}_beta{imspire_param.OptimalTransportParam_Beta}_epsilon{imspire_param.OptimalTransportParam_Epsilon}.npy",ot_solver.T)

#### 3.3.2 one step

In [ ]:
imspire_run=imspire.ImSpiRE(imspire_param)

In [ ]:
imspire_run.run()